In [4]:
#importar as bibliotecas necessárias

import pandas as pd
import requests
import io
from PIL import Image, ImageEnhance, ImageFilter
from rembg import remove

In [7]:
!pip install xlrd

   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   ------------ --------------------------- 30.7/96.5 kB 435.7 kB/s eta 0:00:01
   ------------------------- -------------- 61.4/96.5 kB 544.7 kB/s eta 0:00:01
   ---------------------------------------- 96.5/96.5 kB 612.0 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
#Leitura da planilha

planilha = pd.read_excel('produtos_marcas_site.xls')

In [13]:
planilha.to_excel('produtos_marcas_site.xlsx', index= False)

In [22]:
planilha = pd.read_excel('produtos_marcas_site.xls')

In [23]:
display(planilha)

,ID,COD,Descrição,Unidade,Classificação fiscal,Origem,Preço,Valor IPI fixo,Observações,Situação,...,Dias para preparação,Controlar lotes,Unidade por caixa,URL imagem externa 7,URL imagem externa 8,URL imagem externa 9,URL imagem externa 10,Markup,Permitir inclusão nas vendas,EX TIPI
0,763089936,29,ACETONA 100 AZUL AVANTE COD. 29,UN,2914.11.00,"0 - Nacional, exceto as indicadas nos códigos ...",2.74,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
1,763089941,253,ACETONA 100 ML COD. 253 ROSA AVANT,UN,2914.11.00,"0 - Nacional, exceto as indicadas nos códigos ...",2.74,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
2,763089946,257,ACETONA 500 ML COD. 257 ROSA AVANT,UN,2914.11.00,"0 - Nacional, exceto as indicadas nos códigos ...",12.30,0,NaN,Ativo,...,0,Sim,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
3,763089951,89,ACETONA 500ML ML COD. 89 AZUL AVANT,UN,2914.11.00,"0 - Nacional, exceto as indicadas nos códigos ...",12.30,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
4,763090027,ALGODÃO 100,ALGODÃO HIDROFILO NATHY 100G BOLA,UN,5203.00.00,"0 - Nacional, exceto as indicadas nos códigos ...",4.51,0,NaN,Ativo,...,0,Sim,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,806799955,GLOSSY ME NUDE,GLOSSY ME - BRILHO LABIAL NUDE/ BEGE OCEANE,NaN,NaN,"0 - Nacional, exceto as indicadas nos códigos ...",23.20,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
1357,800850077,"BENIG PRETO 1,5","HENNA PRETO 1,5G LA BENIG",UN,1211.90.90,"0 - Nacional, exceto as indicadas nos códigos ...",10.90,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
1358,801063854,LIXA BLOCO ROSA C/10,LIXA BLOCO DE UNHA ROSA COM 10 UNI,PACOTE,NaN,"0 - Nacional, exceto as indicadas nos códigos ...",24.00,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,1.60,Sim,NaN
1359,799865059,NaN,MINI KIT DE PINCEIS,UN,9603.30.00,"0 - Nacional, exceto as indicadas nos códigos ...",0.00,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,1.55,Sim,NaN


In [28]:
#Funções pra melhorar imagens (mediante o def)

def enhance_image(image):
    if image is not None:
        # Redimensionar a imagem para)
        image = image.resize(int((image.width * 1.8), int(image.height * 1.8)), Image.LANCZOS)
        
        # Aumentar a nitidez
        enhancer = ImageEnhance.Sharpness(image)
        image = enhancer.enhance(2.0)  # O valor 2.0 dobra a nitidez
        
        # Aumentar o brilho
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(1.2)  # O valor 1.2 aumenta o brilho em 20%
        
        # Aumentar o contraste
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.5)  # O valor 1.5 aumenta o contraste em 50%
    
        return image
    else:
        return image

In [29]:
for index, row in planilha.iterrows():
    nome_produto = row['COD']
    url_imagem = row['URL']
    
    # Fazer o download da imagem
    response = requests.get(url_imagem)
    
    try:
        # Criar um objeto de imagem a partir da imagem baixada
        image = Image.open(io.BytesIO(response.content))
        
        # Remover o fundo da imagem
        image_with_transparent_background = remove(image)
        
        # Melhorar a qualidade da imagem
        image_with_transparent_background = enhance_image(image_with_transparent_background)
        
        # Salvar a imagem com o nome do produto como PNG
        nome_arquivo = f"{nome_produto}.png"
        image_with_transparent_background.save(nome_arquivo, format="PNG")
        
        # Resultado:
        print(f"Imagem salva como {nome_arquivo}")
    except Exception as e:
        print(f"Não foi possível baixar a imagem para o produto '{nome_produto}': {e}")

Não foi possível baixar a imagem para o produto '29': int() base must be >= 2 and <= 36, or 0
Não foi possível baixar a imagem para o produto '253': int() base must be >= 2 and <= 36, or 0
Não foi possível baixar a imagem para o produto '257': int() base must be >= 2 and <= 36, or 0


KeyboardInterrupt: 

In [32]:
# Função para melhorar a qualidade da imagem
def enhance_image(image):
    # Verificar se a imagem é válida antes de aplicar as melhorias
    if image is not None:
        # Imprimir as dimensões atuais da imagem
        print(f"Dimensões originais da imagem: {image.width}x{image.height}")
        
        # Redimensionar a imagem (exemplo: aumentar o tamanho para 1.8 vezes)
        # new_width = int(image.width * 1.8)
        # new_height = int(image.height * 1.8)
        # print(f"Dimensões redimensionadas da imagem: {new_width}x{new_height}")
        
        # image = image.resize((new_width, new_height), Image.LANCZOS)
        
        # Aumentar a nitidez
        enhancer = ImageEnhance.Sharpness(image)
        image = enhancer.enhance(2.0)  # O valor 2.0 dobra a nitidez
        
        # Aumentar o brilho
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(1.2)  # O valor 1.2 aumenta o brilho em 20%
        
        # Aumentar o contraste
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.5)  # O valor 1.5 aumenta o contraste em 50%
        
        return image
    else:
        return image

# Iterar sobre cada linha da planilha
for index, row in planilha.iterrows():
    nome_produto = str(row['COD']).strip()  # Garantir que o nome do produto seja uma string
    url_imagem = row['URL']
    
    # Verificar se os valores são válidos
    if pd.isna(nome_produto) or pd.isna(url_imagem):
        print(f"Valores inválidos na linha {index + 1}: 'COD' ou 'URL' estão ausentes")
        continue

    # Fazer o download da imagem
    response = requests.get(url_imagem)
    
    try:
        # Criar um objeto de imagem a partir da imagem baixada
        image = Image.open(io.BytesIO(response.content))
        
        # Remover o fundo da imagem
        image_with_transparent_background = remove(image)
        
        # Melhorar a qualidade da imagem
        image_with_transparent_background = enhance_image(image_with_transparent_background)
        
        # Salvar a imagem com o nome do produto como PNG
        nome_arquivo = f"{nome_produto}.png"
        image_with_transparent_background.save(nome_arquivo, format="PNG")
        
        # Resultado:
        print(f"Imagem salva como {nome_arquivo}")
    except Exception as e:
        print(f"Não foi possível baixar a imagem para o produto '{nome_produto}': {e}")

Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como 29.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como 253.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como 257.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como 89.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como ALGODÃO 100.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como ALGODÃO 250.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como ALGODÃO 25.png
Dimensões originais da imagem: 234x215
Dimensões redimensionadas da imagem: 421x387
Imagem salva como ALGODÃO 25 CX.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Image

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Dimensões originais da imagem: 1200x1200
Dimensões redimensionadas da imagem: 2160x2160
Imagem salva como 33.png
Dimensões originais da imagem: 1540x1540
Dimensões redimensionadas da imagem: 2772x2772
Imagem salva como 80.png
Dimensões originais da imagem: 1200x1200
Dimensões redimensionadas da imagem: 2160x2160
Imagem salva como 213.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como 223.png
Dimensões originais da imagem: 450x450
Dimensões redimensionadas da imagem: 810x810
Imagem salva como 167.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como MCLK332.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como MCLK339.png
Dimensões originais da imagem: 225x225
Dimensões redimensionadas da imagem: 405x405
Imagem salva como MCL3011.png
Dimensões originais da imagem: 500x500
Dimensões redimensionadas da imagem: 900x900
Imagem salva c